## Setup

In [1]:
import numpy as np
import random
import tensorflow as tf
from tqdm import tqdm

In [2]:
# https://www.tensorflow.org/install/pip

In [3]:
# conda install -c conda-forge cudatoolkit=11.2 cudnn=8.1.0
# export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$CONDA_PREFIX/lib/
# python3 -m pip install tensorflow==2.6
# # Verify install:
# python3 -c "import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))"

In [4]:
# tf.test.is_gpu_available(
#     cuda_only=False, min_cuda_compute_capability=None
# )

In [5]:
print(tf.config.list_physical_devices("GPU"))
print("Default GPU Device: {}".format(tf.test.gpu_device_name()))
print(tf.test.is_built_with_cuda())

from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Default GPU Device: /device:GPU:0
True
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14508049985196786461
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 23857266688
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16477633679259566700
physical_device_desc: "device: 0, name: Quadro RTX 6000, pci bus id: 0000:73:00.0, compute capability: 7.5"
]


2022-11-22 15:24:09.644344: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-22 15:24:10.204043: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /device:GPU:0 with 22752 MB memory:  -> device: 0, name: Quadro RTX 6000, pci bus id: 0000:73:00.0, compute capability: 7.5
2022-11-22 15:24:10.206392: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /device:GPU:0 with 22752 MB memory:  -> device: 0, name: Quadro RTX 6000, pci bus id: 0000:73:00.0, compute capability: 7.5


In [6]:
import os
import sys

my_lib_path = os.path.abspath("/home/mai.kassem/Documents/ML701-AilmentClassification/")
sys.path.append(my_lib_path)

import lstm_module

[nltk_data] Downloading package punkt to /home/mai.kassem/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mai.kassem/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/mai.kassem/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/mai.kassem/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


max_length:  13


## Initialize BA parameters

In [7]:
n = 10  # population size
m = 7  # good bees
e = 3  # the elite bees
num = 2  # numbers of the LSTM parameters - number of epochs and number of units
nep = 4
nsp = 2
ngh = 1  # neighborhood size
opt = np.zeros([1, num + 1])  # the result (i.e., the optimal bee)
# second parameter is the LSTM parameters + the accuracy
population = np.zeros([n, num + 1])
good = np.zeros([m - e, num + 1])
elite = np.zeros([e, num + 1])

epochs_start = 10
epochs_end = 50
units_start = 8
units_end = 128

## Helper functions

In [8]:
def generate_bee(index):
    # generate the parameters using uniform distribution function
    # first parameter
    population[index][0] = int(random.uniform(epochs_start, epochs_end))
    # second parameter
    population[index][1] = int(random.uniform(units_start, units_end))

In [9]:
def evaluate(bee):
    # train NN and calculate the accuracy
    model = lstm_module.LSTM(num_epochs=int(bee[0]), num_units=int(bee[1]))

    # without k-fold
    accuracy = lstm_module.evaluate_model(model)
    bee[num] = accuracy

    # with k-fold
    # predicted_target, actual_target, average_accuracy = lstm_module.evaluate_model(model)
    # bee[num] = average_accuracy
    return bee

In [10]:
def sort_bees(unsorted):
    # sort list of bees based on the accuracy column
    return np.array(sorted(unsorted, key=lambda x: x[num], reverse=True))

In [11]:
def local_search(bee, st, amount):
    new_bees = np.zeros([amount, num + 1])
    for i in range(amount):
        # generate new bee in neighborhood of current bee
        nbee = [0, 0, 0]
        # first parameter
        nbee[0] = int(random.uniform(bee[0] - ngh, bee[0] + ngh))
        # second parameter
        nbee[1] = int(random.uniform(bee[1] - ngh, bee[1] - ngh))
        # evaluate new bee
        nbee = evaluate(nbee)
        # if the evaluation of new bee is better than original, keep it else save the original
        if nbee[num] > bee[num]:
            new_bees[i] = nbee
        else:
            new_bees[i] = bee

    new_bees = sort_bees(new_bees)
    return new_bees[0]

## Load data

## BA

In [12]:
lstm_module.print_data()

(2196, 13)
(2196,)
(245, 13)
(245,)
(272, 13)
(272,)
969
13


In [13]:
for i in tqdm(range(n)):
    generate_bee(i)
    # evaluate the population
    population[i] = evaluate(population[i])

# sort the population decesntly based on the evaluation value (i.e., accuracy)
population = sort_bees(population)

# opt=first bee in the population
opt = population[0]

  0%|          | 0/10 [00:00<?, ?it/s]2022-11-22 15:24:22.807235: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22752 MB memory:  -> device: 0, name: Quadro RTX 6000, pci bus id: 0000:73:00.0, compute capability: 7.5
2022-11-22 15:24:23.380858: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-11-22 15:24:25.081923: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8100
100%|██████████| 10/10 [02:40<00:00, 16.08s/it]


In [14]:
print(population)
print()
print(opt)

[[ 49.         119.           0.98897058]
 [ 36.          70.           0.9852941 ]
 [ 40.          72.           0.97426468]
 [ 31.          19.           0.97058821]
 [ 30.          71.           0.97058821]
 [ 14.          66.           0.96691179]
 [ 17.          30.           0.95955884]
 [ 32.          18.           0.94485295]
 [ 14.          74.           0.93382353]
 [ 34.           9.           0.63235295]]

[ 49.         119.           0.98897058]


In [15]:
maxIter = 10
for iter in tqdm(range(maxIter)):
    print("Iteration==========================", iter)
    print(population)

    for i in range(e):
        # call local search for elite and update the population
        population[i] = local_search(population[i], "elite", nep)

    for j in range(i, m):
        # call local search for the remaining good bees and update the population
        population[j] = local_search(population[j], "good", nsp)

    for k in range(j, n):
        # generate the remaining of the population randomly and evaluate
        generate_bee(k)
        # evaluate the population
        population[k] = evaluate(population[k])

    # sort the population
    population = sort_bees(population)
    # opt= the first bee in the population
    opt = population[0]

  0%|          | 0/10 [00:00<?, ?it/s]

Iteration========================== 0
[[ 49.         119.           0.98897058]
 [ 36.          70.           0.9852941 ]
 [ 40.          72.           0.97426468]
 [ 31.          19.           0.97058821]
 [ 30.          71.           0.97058821]
 [ 14.          66.           0.96691179]
 [ 17.          30.           0.95955884]
 [ 32.          18.           0.94485295]
 [ 14.          74.           0.93382353]
 [ 34.           9.           0.63235295]]


 10%|█         | 1/10 [07:23<1:06:27, 443.08s/it]

Iteration========================== 1
[[ 39.          70.           0.99264705]
 [ 49.         119.           0.98897058]
 [ 36.          70.           0.9852941 ]
 [ 29.          70.           0.9852941 ]
 [ 33.          70.           0.97794116]
 [ 10.          83.           0.97426468]
 [ 31.          19.           0.97058821]
 [ 13.          65.           0.97058821]
 [ 30.          15.           0.95955884]
 [ 12.          16.           0.7022059 ]]


 20%|██        | 2/10 [15:09<1:00:56, 457.08s/it]

...

 80%|████████  | 8/10 [1:03:39<15:53, 476.74s/it]

Iteration========================== 8
[[ 47.         117.           0.99632353]
 [ 39.          70.           0.99264705]
 [ 35.          69.           0.99264705]
 [ 47.         117.           0.99264705]
 [ 38.          42.           0.99264705]
 [ 28.          69.           0.98897058]
 [ 36.          21.           0.9852941 ]
 [ 44.          95.           0.97794116]
 [ 10.          65.           0.93014705]
 [ 10.         111.           0.91911763]]


 90%|█████████ | 9/10 [1:11:25<07:53, 473.38s/it]

Iteration========================== 9
[[ 47.         117.           0.99632353]
 [ 39.          70.           0.99264705]
 [ 35.          69.           0.99264705]
 [ 47.         117.           0.99264705]
 [ 38.          42.           0.99264705]
 [ 28.          69.           0.98897058]
 [ 25.         108.           0.98897058]
 [ 21.          79.           0.97058821]
 [ 18.         108.           0.96323532]
 [ 24.         108.           0.95955884]]


100%|██████████| 10/10 [1:19:17<00:00, 475.74s/it]


In [16]:
print(population)
print()
print(opt)

[[ 47.         117.           0.99632353]
 [ 39.          70.           0.99264705]
 [ 35.          69.           0.99264705]
 [ 47.         117.           0.99264705]
 [ 38.          42.           0.99264705]
 [ 28.          69.           0.98897058]
 [ 49.         123.           0.98897058]
 [ 24.         116.           0.98161763]
 [ 40.          77.           0.98161763]
 [ 13.         123.           0.96691179]]

[ 47.         117.           0.99632353]
